## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-16-22-49-36 +0000,nypost,Nassau County breaks its own rules by keeping ...,https://nypost.com/2025/12/16/us-news/nassau-c...
1,2025-12-16-22-48-00 +0000,wsj,Opinion | Where Are Those Manufacturing Jobs?,https://www.wsj.com/opinion/jobs-report-bls-ma...
2,2025-12-16-22-41-58 +0000,nypost,Brad Lander wasn’t allowed to use iconic ‘Mist...,https://nypost.com/2025/12/16/us-news/brad-lan...
3,2025-12-16-22-41-33 +0000,nyt,Sydney Attackers Motivated by “Islamic State I...,https://www.nytimes.com/2025/12/16/world/austr...
4,2025-12-16-22-40-00 +0000,wsj,Opinion | ISIS Rears Its Head Once More,https://www.wsj.com/opinion/isis-syria-austral...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2472/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
98,trump,48
416,bondi,28
40,reiner,21
57,new,19
417,beach,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
122,2025-12-16-19-36-00 +0000,nyt,"Susie Wiles, Trump’s Chief of Staff, Criticize...",https://www.nytimes.com/2025/12/16/us/politics...,126
136,2025-12-16-19-05-48 +0000,nypost,Trump stands by chief of staff Susie Wiles aft...,https://nypost.com/2025/12/16/us-news/trump-st...,114
79,2025-12-16-21-05-32 +0000,nypost,Australia opposition party launches new antise...,https://nypost.com/2025/12/16/world-news/austr...,113
231,2025-12-16-13-33-47 +0000,nypost,White House chief of staff Susie Wiles: AG Pam...,https://nypost.com/2025/12/16/us-news/ag-pam-b...,108
132,2025-12-16-19-17-01 +0000,nypost,Trump thinks Russia’s Putin wants all of Ukrai...,https://nypost.com/2025/12/16/world-news/trump...,108


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
122,126,2025-12-16-19-36-00 +0000,nyt,"Susie Wiles, Trump’s Chief of Staff, Criticize...",https://www.nytimes.com/2025/12/16/us/politics...
126,89,2025-12-16-19-28-46 +0000,bbc,"What we know about Nick Reiner, son arrested i...",https://www.bbc.com/news/articles/cvgj14l3v8vo...
79,85,2025-12-16-21-05-32 +0000,nypost,Australia opposition party launches new antise...,https://nypost.com/2025/12/16/world-news/austr...
258,60,2025-12-16-11-07-53 +0000,bbc,Dashcam video shows couple trying to stop gunm...,https://www.bbc.com/news/articles/cx2341yx719o...
107,49,2025-12-16-20-03-00 +0000,wsj,"The Pentagon won’t release the full, unedited ...",https://www.wsj.com/politics/national-security...
89,44,2025-12-16-20-33-00 +0000,wsj,"In a two-part series published in Vanity Fair,...",https://www.wsj.com/politics/policy/susie-wile...
84,39,2025-12-16-20-47-00 +0000,wsj,Russia-Ukraine Peace Efforts Send Crude to Nea...,https://www.wsj.com/business/oil-falls-amid-pr...
65,38,2025-12-16-21-28-19 +0000,nypost,Timeline showing movements of Brown University...,https://nypost.com/2025/12/16/us-news/timeline...
15,36,2025-12-16-22-25-14 +0000,nypost,Jeffrey Epstein kept Bear Stearns job despite ...,https://nypost.com/2025/12/16/business/jeffrey...
209,35,2025-12-16-15-02-07 +0000,nypost,Bondi Beach terrorists had homemade bombs and ...,https://nypost.com/2025/12/16/world-news/bondi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
